In [2]:
from atmpy.flux.flux import Flux
from atmpy.grid.kgrid import Grid
from atmpy.variables.variables import Variables
import numpy as np
from atmpy.grid.utility import DimensionSpec, create_grid
from atmpy.physics.eos import ExnerBasedEOS
from atmpy.data.enums import PrimitiveVariableIndices as PVI, VariableIndices as VI


np.set_printoptions(linewidth=100)

dt = 0.1

dim = [DimensionSpec(996, 0, 2, 2), DimensionSpec(996, 0, 2, 2)]
grid = create_grid(dim)
rng = np.random.default_rng()
arr = np.arange(1_000_000)
rng.shuffle(arr)
array = arr.reshape(1000, 1000)

variables = Variables(grid, 5, 1)
variables.cell_vars[..., VI.RHO] = 1
variables.cell_vars[..., VI.RHOU] = array
variables.cell_vars[..., VI.RHOY] = 2

rng.shuffle(arr)
array = arr.reshape(1000, 1000)
variables.cell_vars[..., VI.RHOV] = array
eos = ExnerBasedEOS()
flux = Flux(grid, variables, eos, dt)
variables.to_primitive(eos)
primitives = variables.primitives
direction = "x"

# %time flux.apply_riemann_solver(1, direction, True)
%time flux.apply_riemann_solver(1, direction)

CPU times: user 235 ms, sys: 107 ms, total: 342 ms
Wall time: 310 ms
